In [80]:
import torchvision
from torchvision import transforms
import torch
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ExponentialLR, CosineAnnealingLR
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from BaseModel import CustomCNN
import torch.nn as nn
import torch
import torch.optim as optim
import json
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import pandas as pd


### Import hyperparameter ###

In [83]:
with open('D:\DPL302m\project\Abnormal_Behavior\Model\param.json', 'r') as json_file:
    data = json.load(json_file)

for key, value in data.items():
    globals()[key] = value

### Load dataset from numpy file and transform to tensor

In [84]:
def LoadData(name=""):
    data = np.load(name+".npz")
    images = data['images']
    labels = data['labels']

    images = torch.from_numpy(images)
    labels = torch.from_numpy(labels)

    dataset = TensorDataset(images, labels)
    return dataset


### Load CustomCNN model from BaseModel file

In [1]:
from BaseModel import CustomCNN

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration 

In [85]:
#Config

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('DEVICE:',DEVICE)
model = CustomCNN(num_classes=2)
criterion = nn.CrossEntropyLoss()
model.to(DEVICE)
criterion.to(DEVICE)
batch_size = BATCH_SIZE
EPOCHS = 10

optimizer = optim.Adam(model.parameters(), lr=float(LEARNING_RATE))
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=7)

TRAINLOADER = DataLoader(LoadData('Data_train'), batch_size=batch_size, shuffle=True)
TESTLOADER = DataLoader(LoadData('Data_test'), batch_size=batch_size, shuffle=False)


device(type='cpu')

## Training

In [ ]:
performance= []
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    predictions_train = []
    true_labels_train = []
    
    for inputs, labels in TRAINLOADER:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE).long()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss_train = criterion(outputs, labels)
        loss_train.backward()
        optimizer.step()

        running_loss += loss_train.item()
        _, predicted = torch.max(outputs.data, 1)
        predictions_train.extend(predicted.tolist())
        true_labels_train.extend(labels.tolist())

    train_f1 = f1_score(true_labels_train, predictions_train, average='weighted')

    if scheduler is not None:
        scheduler.step()

    model.eval() 
    total_loss_test = 0.0
    total_samples = 0
    predictions_test = []
    true_labels_test = []
    with torch.no_grad():
        for inputs, labels in TESTLOADER:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE).long()
            outputs = model(inputs)
            loss_test = criterion(outputs, labels)
            total_loss_test += loss_test.item() * len(labels)
            total_samples += len(labels)

            _, predicted = torch.max(outputs.data, 1)
            predictions_test.extend(predicted.tolist())
            true_labels_test.extend(labels.tolist())

    avg_loss_test = total_loss_test / total_samples
    test_f1 = f1_score(true_labels_test, predictions_test, average='weighted')
    performance.append([running_loss / len(TRAINLOADER),avg_loss_test,train_f1,test_f1])
    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss Train: {performance[epoch][0] :.4f} Loss Test: {performance[epoch][1]:.4f} F1 Train: {performance[epoch][2]:.4f} F1 Test: {performance[epoch][3] :.4f}")
#Save Performance
df = pd.DataFrame(performance, columns=['Loss Train', 'Loss Test', 'F1 Train','F1 Test'])
df.to_csv('Performance.csv',index=False)